### 벡터 DB 기반의 Retriever

#### 환경설정

In [1]:
!pip install -q langchain tiktoken langchain-text-splitters langchain-openai langchain-chroma pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.1 MB/s eta 0:

In [2]:
# 01 colab용 비밀 키 불러오기
from google.colab import userdata
import os
from openai import OpenAI # openai-python SDK

# 02 OpenAI API키 설정
api_key = userdata.get("modulabsKey") # colab 사용자 환경변수 탭에서 등록한 키
os.environ["OPEN_API_KEY"] = api_key
client = OpenAI(api_key = api_key)

# 03 버전 확인(선택)
import openai
import langchain
print("LangChain ver.",langchain.__version__)
print("OpenAI SDK ver.", openai.__version__)

LangChain ver. 0.3.26
OpenAI SDK ver. 1.91.0


#### 01 벡터DB 기반 Retriever
- Chroma 벡터 DB 기반 기본 유사 문서 검색

In [3]:
!pip install --quiet langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00


In [5]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

import os
Chroma().delete_collection() # 기존에 정의한 collection 삭제

# 앞에서 정의했지만 명시적으로 적어둠
api_key = userdata.get("modulabsKey")
os.environ["OPENAI_API_KEY"] = api_key

# colab mount
from google.colab import drive
drive.mount('/content/drive')

# 헌법 PDF 파일 로드
loader = PyPDFLoader(r"/content/drive/MyDrive/ColabNotebooks/03_Modulabs/Modu_LLM/data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

# PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

# ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(
    model = 'text-embedding-3-small'
))

# Chroma를 Retriever로 활용
retriever = db.as_retriever()
retriever.invoke("국회의원의 의무")

Mounted at /content/drive


[Document(id='19834ec4-01d9-4a0a-9118-766245f24511', metadata={'page_label': '5', 'total_pages': 14, 'moddate': '2024-04-01T21:26:24+09:00', 'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2024-04-01T21:26:24+09:00', 'source': '/content/drive/MyDrive/ColabNotebooks/03_Modulabs/Modu_LLM/data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'page': 4}, page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.'),
 Document(id='c9a69f17-ecbb-4de6-83c6-9fd0232ef4af', metadata={'producer': 'iText 2.1.7 by 1T3XT', 'page_label

##### (1) 유사도 점수도 함께 출력하기

In [6]:
result_score = db.similarity_search_with_score("국회의원의 의무")
result_r_score = db.similarity_search_with_relevance_scores("국회의원의 의무")
print("[유사 청크 1순위]")
print(result_score[0][0].page_content)
print("\n\n[점수]")
print(result_score[0][1])
print(result_r_score[0][1])

[유사 청크 1순위]
③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이
익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.
 
제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원
4분의 1 이상의 요구에 의하여 집회된다.
②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.
③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.
 
제48조 국회는 의장 1인과 부의장 2인을 선출한다.
 
제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결
한다. 가부동수인 때에는 부결된 것으로 본다.
 
제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다
고 인정할 때에는 공개하지 아니할 수 있다.


[점수]
0.8001738786697388
0.43392453968527256


##### (2) 검색 결과 수 및 조정

In [7]:
#유사 청크 1개만 반환
retriever = db.as_retriever(search_kwargs={"k": 1})
retriever.invoke("국회의원의 의무")

[Document(id='19834ec4-01d9-4a0a-9118-766245f24511', metadata={'source': '/content/drive/MyDrive/ColabNotebooks/03_Modulabs/Modu_LLM/data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'page': 4, 'total_pages': 14, 'producer': 'iText 2.1.7 by 1T3XT', 'creationdate': '2024-04-01T21:26:24+09:00', 'page_label': '5', 'moddate': '2024-04-01T21:26:24+09:00', 'creator': 'PyPDF'}, page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.')]

##### (3) 검색 방식 변경 - MMR

In [8]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

#헌법 PDF 파일 로드
loader = PyPDFLoader(r"/content/drive/MyDrive/ColabNotebooks/03_Modulabs/Modu_LLM/data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))

###### 01 MMR검색방식
- lambda값을 0으로 지정, 유사도만 고려함!

In [9]:
#Chroma를 Retriever로 활용
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs = {"lambda_mult": 0, "fetch_k":10, "k":3}
)
retriever.invoke("국회의원의 의무")

[Document(id='19834ec4-01d9-4a0a-9118-766245f24511', metadata={'page_label': '5', 'page': 4, 'creationdate': '2024-04-01T21:26:24+09:00', 'moddate': '2024-04-01T21:26:24+09:00', 'total_pages': 14, 'creator': 'PyPDF', 'source': '/content/drive/MyDrive/ColabNotebooks/03_Modulabs/Modu_LLM/data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'producer': 'iText 2.1.7 by 1T3XT'}, page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.'),
 Document(id='35b24af2-f6ec-41ba-95bc-1b16f0d96c28', metadata={'page': 5, 'creationdate': '2024-04-01T21:26:24

###### 02 일반 유사도 검색

In [10]:
# Chroma를 Retriever로 활용
retriever = db.as_retriever(search_kwargs = {"k":3})
retriever.get_relevant_documents("국회의원의 의무")

/tmp/ipython-input-10-878389895.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("국회의원의 의무")


[Document(id='19834ec4-01d9-4a0a-9118-766245f24511', metadata={'total_pages': 14, 'producer': 'iText 2.1.7 by 1T3XT', 'creationdate': '2024-04-01T21:26:24+09:00', 'source': '/content/drive/MyDrive/ColabNotebooks/03_Modulabs/Modu_LLM/data/대한민국헌법(헌법)(제00010호)(19880225).pdf', 'page_label': '5', 'page': 4, 'moddate': '2024-04-01T21:26:24+09:00', 'creator': 'PyPDF'}, page_content='③국회의원은 그 지위를 남용하여 국가ㆍ공공단체 또는 기업체와의 계약이나 그 처분에 의하여 재산상의 권리ㆍ이\n익 또는 직위를 취득하거나 타인을 위하여 그 취득을 알선할 수 없다.\n \n제47조 ①국회의 정기회는 법률이 정하는 바에 의하여 매년 1회 집회되며, 국회의 임시회는 대통령 또는 국회재적의원\n4분의 1 이상의 요구에 의하여 집회된다.\n②정기회의 회기는 100일을, 임시회의 회기는 30일을 초과할 수 없다.\n③대통령이 임시회의 집회를 요구할 때에는 기간과 집회요구의 이유를 명시하여야 한다.\n \n제48조 국회는 의장 1인과 부의장 2인을 선출한다.\n \n제49조 국회는 헌법 또는 법률에 특별한 규정이 없는 한 재적의원 과반수의 출석과 출석의원 과반수의 찬성으로 의결\n한다. 가부동수인 때에는 부결된 것으로 본다.\n \n제50조 ①국회의 회의는 공개한다. 다만, 출석의원 과반수의 찬성이 있거나 의장이 국가의 안전보장을 위하여 필요하다\n고 인정할 때에는 공개하지 아니할 수 있다.'),
 Document(id='98d0d759-83e7-4009-b11f-b0d758c0983a', metadata={'page_label': '5', 'moddate': '2024-04-01T21:26

#### 02 MultiQueryRetriever
- 사용자의 쿼리를 재해석해 검색하는 MultiQueryRetriever

##### Chroma DB에 문서 벡터 저장

In [11]:
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

loaders = PyPDFLoader(r"/content/drive/MyDrive/ColabNotebooks/03_Modulabs/Modu_LLM/data/대한민국헌법(헌법)(제00010호)(19880225).pdf")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
docs = text_splitter.split_documents(docs)

##### Multi Vector를 만들기 위한 작업

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

vectorstore = Chroma(
    collection_name="full_documents", embedding_function=embedding
)
# 상위 문서 저장 위한 레이어 선언
store = InMemoryByteStore()
id_key = "doc_id"
# 상위 문서와 하위 문서를 연결할 키값으로 doc_id 사용
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

#문서 id로 고유한 값을 지정하기 위해 uuid 라이브러리 호출
import uuid
doc_ids = [str(uuid.uuid4()) for _ in docs]

/tmp/ipython-input-12-2266624504.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
# 하위 청크로 쪼개기 위한 child_text_splitter 지정
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# 상위 청크들을 순회하며 하위 청크로 분할한 후 상위 청크 id 상속
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)
#vectorstore에 하위 청크 추가
retriever.vectorstore.add_documents(sub_docs)
#docstore에 상위청크 저장할 때, doc_ids 지정
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [14]:
# Vectorstore alone retrieves the small chunks
print("[하위 청크] \n")
print(retriever.vectorstore.similarity_search("국민의 권리")[0].page_content)
print("-"*50)
print("[상위 청크] \n")
print(retriever.invoke("국민의 권리")[0].page_content)

[하위 청크] 

③공공필요에 의한 재산권의 수용ㆍ사용 또는 제한 및 그에 대한 보상은 법률로써 하되, 정당한 보상을 지급하여야
한다.
 
제24조 모든 국민은 법률이 정하는 바에 의하여 선거권을 가진다.
 
제25조 모든 국민은 법률이 정하는 바에 의하여 공무담임권을 가진다.
 
제26조 ①모든 국민은 법률이 정하는 바에 의하여 국가기관에 문서로 청원할 권리를 가진다.
②국가는 청원에 대하여 심사할 의무를 진다.
 
제27조 ①모든 국민은 헌법과 법률이 정한 법관에 의하여 법률에 의한 재판을 받을 권리를 가진다.
②군인 또는 군무원이 아닌 국민은 대한민국의 영역 안에서는 중대한 군사상 기밀ㆍ초병ㆍ초소ㆍ유독음식물공급
ㆍ포로ㆍ군용물에 관한 죄중 법률이 정한 경우와 비상계엄이 선포된 경우를 제외하고는 군사법원의 재판을 받지
--------------------------------------------------
[상위 청크] 

법제처                                                            3                                                       국가법령정보센터
대한민국헌법
제21조 ①모든 국민은 언론ㆍ출판의 자유와 집회ㆍ결사의 자유를 가진다.
②언론ㆍ출판에 대한 허가나 검열과 집회ㆍ결사에 대한 허가는 인정되지 아니한다.
③통신ㆍ방송의 시설기준과 신문의 기능을 보장하기 위하여 필요한 사항은 법률로 정한다.
④언론ㆍ출판은 타인의 명예나 권리 또는 공중도덕이나 사회윤리를 침해하여서는 아니된다. 언론ㆍ출판이 타인의
명예나 권리를 침해한 때에는 피해자는 이에 대한 피해의 배상을 청구할 수 있다.
 
제22조 ①모든 국민은 학문과 예술의 자유를 가진다.
②저작자ㆍ발명가ㆍ과학기술자와 예술가의 권리는 법률로써 보호한다.
 
제23조 ①모든 국민의 재산권은 보장된다. 그 내용과 한계는 법률로 정한다.
②재산권의 행사는 공공복리에 적합하도록 하여야 한다.
③공공필요에 의한 재산권의 수용

#### 03 Long-Context Reorder
- 최근 LLM은('25년기준) GPT-4의 경우 128,000토큰, Claude-3은 200,000토큰 지원하여 대량의 컨텍스트 주입 가능
- 컨텍스트 윈도우 늘어남에 따라, 'Needle in a haystack'(건초더미에서 바늘 찾기)라는 평가지표가 주목받게 됨

##### (1) Long-Context Reorder 없음

In [17]:
# Chroma dimension 관련 에러 발생 시 실행
# Chroma().delete_collection()

In [18]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.document_transformers import(
    LongContextReorder,
)

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_openai import OpenAI

# 한글 임베딩 모델 선언
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_openai import OpenAI

# 한글 임베딩 모델 선언
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

texts = [
    "바스켓볼은 훌륭한 스포츠입니다.",
    "플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.",
    "셀틱스는 제가 가장 좋아하는 팀입니다.",
    "이것은 보스턴 셀틱스에 관한 문서입니다."
    "저는 단순히 영화 보러 가는 것을 좋아합니다.",
    "보스턴 셀틱스가 20점차로 이겼어요",
    "이것은 그냥 임의의 텍스트입니다.",
    "L.코넷은 최고의 셀틱스 선수 중 한 명입니다."
    "래리 버드는 상징적인 NBA 선수였습니다.",

]

# Chroma Retriever 선언(10개의 유사 문서 출력)
retriever = Chroma.from_texts(texts, embedding = embedding).as_retriever(
    search_kwargs = {"k":10}
)

query = "셀틱에 대해 설명해줘"

# 유사도 기준으로 검색 결과 출력
docs = retriever.get_relevant_documents(query)
docs

[Document(id='2cce664e-58d5-498d-a2ab-9a81cf2a7cf2', metadata={}, page_content='L.코넷은 최고의 셀틱스 선수 중 한 명입니다.래리 버드는 상징적인 NBA 선수였습니다.'),
 Document(id='fc3e089c-fd16-45d8-85e9-c6337ed2648f', metadata={}, page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(id='603941c7-70f2-46ab-9b42-5d8b5cd2cef1', metadata={}, page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(id='03d20bb2-8fb4-4bfe-9ba4-1463058441e2', metadata={}, page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다.'),
 Document(id='f1a653e9-b470-45ff-acad-06da8ad5896d', metadata={}, page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(id='c557bca8-4efc-4492-8141-039d90683383', metadata={}, page_content='보스턴 셀틱스가 20점차로 이겼어요'),
 Document(id='9456515d-6f0c-4b51-b164-2a87150a83fd', metadata={}, page_content='플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.')]

##### (2) Long-Context Reorder 활용

In [20]:
# Reorder the documents:
# LongContextReorder 선언
reordering = LongContextReorder()

# 검색된 유사 문서 중 관련도가 높은 문서를 맨 앞에 맨 뒤에 재배치
reordered_docs = reordering.transform_documents(docs)
reordered_docs

[Document(id='2cce664e-58d5-498d-a2ab-9a81cf2a7cf2', metadata={}, page_content='L.코넷은 최고의 셀틱스 선수 중 한 명입니다.래리 버드는 상징적인 NBA 선수였습니다.'),
 Document(id='603941c7-70f2-46ab-9b42-5d8b5cd2cef1', metadata={}, page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(id='f1a653e9-b470-45ff-acad-06da8ad5896d', metadata={}, page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(id='9456515d-6f0c-4b51-b164-2a87150a83fd', metadata={}, page_content='플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.'),
 Document(id='c557bca8-4efc-4492-8141-039d90683383', metadata={}, page_content='보스턴 셀틱스가 20점차로 이겼어요'),
 Document(id='03d20bb2-8fb4-4bfe-9ba4-1463058441e2', metadata={}, page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다.'),
 Document(id='fc3e089c-fd16-45d8-85e9-c6337ed2648f', metadata={}, page_content='셀틱스는 제가 가장 좋아하는 팀입니다.')]